In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install focal_loss

In [ ]:
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt
from pandas import set_option
import pandas as pd
import tensorflow
from tensorflow.keras.models import Sequential
import tensorflow.keras.utils
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, LSTM
import numpy as np 
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sn 
import tensorflow.keras
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import model_from_json
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Conv1D, GlobalAveragePooling1D, MaxPooling1D
from tensorflow.keras import regularizers

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense, Flatten, Conv1D
from tensorflow.keras import backend as K

from tensorflow.keras.layers import Dropout
from sklearn import preprocessing
from sklearn import decomposition
from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report

import tensorflow as tf
import time as tm
import datetime
import os
from operator import itemgetter
from numpy import argmax
from numpy import round
from focal_loss import BinaryFocalLoss

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [ ]:
pip uninstall tensorflow

In [ ]:
pip install tensorflow==2.2.0

In [ ]:
pip show tensorflow

In [ ]:
pip uninstall scikit-learn

In [ ]:
pip install scikit-learn==0.24.0

In [ ]:
pip show scikit-learn

In [ ]:
y = np.load("/content/drive/Shareddrives/Experimentos_CNN/2_labels_DB.fa.filtered_center.npy")
y = y.astype('bool')

In [ ]:
x = np.load("/content/drive/Shareddrives/Experimentos_CNN/2_labels_DB.fasta.filtered_center.npy")
x = x.astype('bool')

In [ ]:
validation_size = 0.2
seed = 7
X_train, X_test_dev, Y_train, Y_test_dev = train_test_split(x, y, test_size=validation_size,random_state=seed)
X_dev, X_test, Y_dev, Y_test = train_test_split(X_test_dev, Y_test_dev, test_size=0.5, random_state=seed)

In [ ]:
X_train.shape

In [ ]:
path_log_base = '/content/drive/Shareddrives/Experimentos_CNN/Estiven/8/logs'

In [ ]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
def metrics(Y_validation,predictions):
    classes = len(np.unique(Y_validation))
    print('Accuracy:', accuracy_score(Y_validation, predictions))
    print('F1 score:', f1_score(Y_validation, predictions,average='weighted'))
    print('Recall:', recall_score(Y_validation, predictions,average='weighted'))
    print('Precision:', precision_score(Y_validation, predictions, average='weighted'))
    print('\n clasification report:\n', classification_report(Y_validation, predictions))
    print('\n confusion matrix:\n',confusion_matrix(Y_validation, predictions))
    #Creamos la matriz de confusión
    snn_cm = confusion_matrix(Y_validation, predictions)

    # Visualizamos la matriz de confusión
    snn_df_cm = pd.DataFrame(snn_cm, range(classes), range(classes))  
    plt.figure(figsize = (20,14))  
    sn.set(font_scale=1.4) #for label size  
    sn.heatmap(snn_df_cm, annot=True, annot_kws={"size": 25},fmt='',cmap="YlOrRd") # font size
    plt.show()

In [ ]:
def SENMAP():
    tf.keras.backend.clear_session()

    # Inputs
    inputs = tf.keras.Input(shape=(X_train.shape[1], X_train.shape[2], 1), name="input_1")
    
    # layer 1
    layers = tf.keras.layers.Conv2D(32, (5, 51),strides=(1, 1) ,activation=tf.keras.layers.LeakyReLU(0.01), kernel_regularizer=regularizers.l1_l2(0.0000, 0.000), bias_regularizer=regularizers.l1_l2(0.0000, 0.0000), use_bias=True)(inputs)
    layers = tf.keras.layers.SpatialDropout2D(0.2)(layers)
    layers = tf.keras.layers.AveragePooling2D((1, 9), strides=None)(layers)
    layers = tf.keras.layers.BatchNormalization(axis=-1, momentum=0.6, epsilon=0.001, scale=False)(layers)

    # layer 2
    layers = tf.keras.layers.Conv2D(64, (1, 31),strides=(1, 1), activation=tf.keras.layers.LeakyReLU(0.01),  kernel_regularizer=regularizers.l1_l2(0.0000, 0.000), bias_regularizer=regularizers.l1_l2(0.0000, 0.0000), use_bias=True)(layers)
    layers = tf.keras.layers.SpatialDropout2D(0.2)(layers)
    layers = tf.keras.layers.AveragePooling2D((1, 9), strides=None)(layers)
    layers = tf.keras.layers.BatchNormalization(axis=1, momentum=0.6, epsilon=0.001, scale=False)(layers)

    # layer 3
    layers = tf.keras.layers.Conv2D(128, (1, 11),strides=(1, 1), activation=tf.keras.layers.LeakyReLU(0.01),  kernel_regularizer=regularizers.l1_l2(0.0000, 0.000), bias_regularizer=regularizers.l1_l2(0.0000, 0.0000), use_bias=True)(layers)
    layers = tf.keras.layers.SpatialDropout2D(0.2)(layers)
    layers = tf.keras.layers.AveragePooling2D((1, 7), strides=None)(layers)
    layers = tf.keras.layers.BatchNormalization(axis=1, momentum=0.6, epsilon=0.001, scale=False)(layers)
    
    # layer 4
    layers = tf.keras.layers.Conv2D(256, (1, 5),strides=(1, 1), activation=tf.keras.layers.LeakyReLU(0.01),  kernel_regularizer=regularizers.l1_l2(0.0000, 0.000), bias_regularizer=regularizers.l1_l2(0.0000, 0.0000), use_bias=True)(layers)
    layers = tf.keras.layers.SpatialDropout2D(0.2)(layers)
    layers = tf.keras.layers.AveragePooling2D((1, 5), strides=None)(layers)
    layers = tf.keras.layers.BatchNormalization(axis=1, momentum=0.6, epsilon=0.001, scale=False)(layers)

    # layer 5
    layers = tf.keras.layers.Flatten()(layers)

    # layer 6
    layers = tf.keras.layers.Dense(300,activation=tf.keras.layers.LeakyReLU(0.01), kernel_regularizer=regularizers.l1_l2(0.0003, 0.001), bias_regularizer=regularizers.l1(0.001))(layers)
    layers = tf.keras.layers.Dropout(0.2)(layers)
    layers = tf.keras.layers.BatchNormalization(momentum=0.6, epsilon=0.001, scale=False)(layers)

    # layer 7
    layers = tf.keras.layers.Dense(300,activation=tf.keras.layers.LeakyReLU(0.01), kernel_regularizer=regularizers.l1_l2(0.0003, 0.001), bias_regularizer=regularizers.l1(0.001))(layers)
    layers = tf.keras.layers.Dropout(0.2)(layers)
    layers = tf.keras.layers.BatchNormalization(momentum=0.6, epsilon=0.001, scale=False)(layers)

    # layer 8
    layers = tf.keras.layers.Dense(300,activation=tf.keras.layers.LeakyReLU(0.01), kernel_regularizer=regularizers.l1_l2(0.0003, 0.001), bias_regularizer=regularizers.l1(0.001))(layers)
    layers = tf.keras.layers.Dropout(0.2)(layers)
    layers=tf.keras.layers.BatchNormalization(momentum=0.6, epsilon=0.001,  scale=False)(layers)

    # layer end
    predictions = tf.keras.layers.Dense(2, activation="softmax", name="output_1")(layers)
    
    # model generation
    model = tf.keras.Model(inputs = inputs, outputs=predictions)
    
    # optimizer
    opt = tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    
    # loss function
    loss_fn = BinaryFocalLoss(gamma=2)
    
    # Compile model
    model.compile(loss=loss_fn, optimizer=opt, metrics=[f1_m])
    return model

In [ ]:
def run_experiment(model):
    optimizer = tfa.optimizers.AdamW(
        learning_rate=learning_rate, weight_decay=weight_decay
    )

    model.compile(
        optimizer=optimizer,
        #loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        loss = BinaryFocalLoss(gamma=2),
        #loss=tf.keras.losses.BinaryFocalCrossentropy(gamma=2.0, from_logits=True),
        #loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
        metrics=[
            keras.metrics.SparseCategoricalAccuracy(name="accuracy"),
            #keras.metrics.SparseTopKCategoricalAccuracy(5, name="top-5-accuracy"),
            #keras.metrics.CategoricalAccuracy(name="categorical_accuracy", dtype=None),
            #keras.metrics.BinaryAccuracy(name='binary_accuracy'),
        ],
    )

    checkpoint_filepath = "checkpoint"
    checkpoint_callback = keras.callbacks.ModelCheckpoint(
        checkpoint_filepath,
        monitor="val_f1_m",
        save_best_only=True,
        save_weights_only=True,
    )

    history = model.fit(
        #x=X_train,
        y=one_hot_labels_train,
        y = Y_train,
        batch_size=batch_size,
        epochs=num_epochs,
        validation_data=(X_dev, one_hot_labels_validation),verbose=1)
        #validation_data=(X_dev, Y_dev),verbose=1)

    return history

In [ ]:
one_hot_labels_train = tf.keras.utils.to_categorical(Y_train, num_classes=2)
one_hot_labels_validation = tf.keras.utils.to_categorical(Y_dev, num_classes=2)
one_hot_labels_test = tf.keras.utils.to_categorical(Y_test, num_classes=2)

In [ ]:
model = SENMAP()
model.summary()
history = run_experiment(model)


acc=history.history['accuracy']
np.save('/home/rtabares/Transformer/accuracy_SENMAP_completeDB.npy',acc)
val_acc=history.history['val_accuracy']
np.save('/home/rtabares/Transformer/val_accuracy_SENMAP_completeDB.npy',val_acc)
loss=history.history['loss']
np.save('/home/rtabares/Transformer/loss_SENMAP_completeDB.npy',loss)
val_loss=history.history['val_loss']
np.save('/home/rtabares/Transformer/val_los_SENMAP_completeDBnpy',val_loss)

In [ ]:
plt.plot(history.history['val_accuracy'])
plt.plot(history.history['accuracy'])
plt.legend(['val_accuracy','train_accuracy'], loc='lower right')
plt.xlabel('Epoch')
plt.ylabel('accuracy')
plt.title('Epoch vs accuracy')
plt.show()
plt.savefig('/home/rtabares/Transformer/Train_Curve_80000.png',  bbox_inches='tight',dpi=500)

In [ ]:
scores = model.evaluate(X_train, one_hot_labels_train, verbose=0)
print("Baseline Error train: %.2f%%" % (100-scores[1]*100))

scores = model.evaluate(X_dev, one_hot_labels_validation, verbose=0)
print("Baseline Error dev: %.2f%%" % (100-scores[1]*100))

scores = model.evaluate(X_test, one_hot_labels_test, verbose=0)
print("Baseline Error test: %.2f%%" % (100-scores[1]*100))

predictions = model.predict(X_train)
metrics(Y_train, [argmax(x) for x in predictions])

predictions = model.predict(X_dev)
metrics(Y_dev, [argmax(x) for x in predictions])

predictions = model.predict(X_test)
metrics(Y_test, [argmax(x) for x in predictions])

In [ ]:
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=1).split(X_train, Y_train)

scores=[]
for k, (train,test) in enumerate(kfold):
  #tf.keras.backend.clear_session()
  model = SENMAP()
  compile_model(model)
  model.fit(X_train[train], Y_train[train], batch_size=128, epochs=200, verbose=0)
  score = accuracy_score(Y_train[test], [argmax(x) for x in model.predict(X_train[test])])
  scores.append(score)

print('Cv accuracy: %.3f +/- %.3f' % (np.mean(scores), np.std(scores)))

print(scores)
np.save('/home/rtabares/Transformer/scores_SENMAP_completeDB.npy',scores)


#Informacion resultado

table = pd.DataFrame(columns=['Predicciones','Salidas_Esperadas','Resultado'])
pred = [argmax(x) for x in predictions]
table['Predicciones'] = pred
table['Salidas_Esperadas'] = Y_test

for i in range(len(pred)):
  if table['Predicciones'][i] == 1:
    table['Predicciones'][i] = True
  else:
    table['Predicciones'][i] = False

for j in range(len(pred)):
  if table['Predicciones'][j] and table['Salidas_Esperadas'][j]:
    table['Resultado'][j] = True
  else:
    table['Resultado'][j] = False
np.save('/home/rtabares/Transformer/SENMAP_predicciones_completeDB.npy',table)
table = table.drop(table[table['Resultado']==True].index)
#print(table_fil.index)
np.save('/home/rtabares/Transformer/SENMAP_indices_completeDB.npy',table.index)